In [33]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [34]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

import os
import numpy as np

import tensorflow as tf
import datetime

Clear any logs from previous runs
!rm -rf ./logs/

In [35]:
#Filter out corrupted images
num_skipped = 0
for folder_name in ("dress", "outer", "pants", "skirt", "suit", "top"):
    folder_path = os.path.join("dataset", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        fobj = open(fpath, "rb")
        if tf.compat.as_bytes("JFIF") not in fobj.peek(10):
            num_skipped += 1
            # Delete corrupted image
            os.system("rm " + fpath)
print("Deleted %d images" % num_skipped)

Deleted 6 images


In [36]:
#Generate a Dataset
image_size = (225, 225)
batch_size = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 22105 files belonging to 6 classes.
Using 17684 files for training.
Found 22105 files belonging to 6 classes.
Using 4421 files for validation.


In [37]:
model = keras.Sequential()

#model.add(layers.experimental.preprocessing.Rescaling(1.0 / 255))

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(225, 225, 3), padding='same' ))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3)))

model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same' ))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(3, 3)))

for size in [128, 256, 512, 728]:
    model.add(layers.Activation("relu"))
    model.add(layers.Conv2D(size, 3, padding="same"))#
    model.add(layers.BatchNormalization())
    
    model.add(layers.Activation("relu"))
    model.add(layers.Conv2D(size, 3, padding="same"))#
    model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D(3, strides=2, padding="same"))

model.add(layers.Conv2D(1024, 3, padding="same"))#
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))

model.add(layers.GlobalAveragePooling2D())

#model.add(layers.Flatten())
#model.add(layers.Dense(256, activation='relu'))

model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

In [38]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 225, 225, 32)      896       
_________________________________________________________________
batch_normalization_33 (Batc (None, 225, 225, 32)      128       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 75, 75, 64)        18496     
_________________________________________________________________
batch_normalization_34 (Batc (None, 75, 75, 64)        256       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
activation_27 (Activation)   (None, 25, 25, 64)       

In [39]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + time
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [41]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [42]:
epochs = 10

model.fit(train_ds, epochs=epochs, callbacks=[tensorboard_callback, earlystopping], validation_data=val_ds, batch_size = 16)

Epoch 1/10
1106/1106 [==============================] - 60s 54ms/step - loss: 1.9603 - accuracy: 0.2113 - val_loss: 1.9070 - val_accuracy: 0.2640
Epoch 2/10
1106/1106 [==============================] - 60s 54ms/step - loss: 1.6940 - accuracy: 0.3017 - val_loss: 1.6556 - val_accuracy: 0.3273
Epoch 3/10
1106/1106 [==============================] - 59s 53ms/step - loss: 1.5149 - accuracy: 0.3899 - val_loss: 1.6242 - val_accuracy: 0.3796
Epoch 4/10
1106/1106 [==============================] - 60s 54ms/step - loss: 1.3717 - accuracy: 0.4607 - val_loss: 1.5153 - val_accuracy: 0.3852
Epoch 5/10
1106/1106 [==============================] - 59s 54ms/step - loss: 1.2501 - accuracy: 0.5173 - val_loss: 1.2673 - val_accuracy: 0.5166
Epoch 6/10
1106/1106 [==============================] - 60s 54ms/step - loss: 1.1462 - accuracy: 0.5656 - val_loss: 1.3747 - val_accuracy: 0.4915
Epoch 7/10
1106/1106 [==============================] - 59s 53ms/step - loss: 1.0617 - accuracy: 0.6040 - val_loss: 1.3099 -

In [43]:
if not os.path.exists('models/'):
        os.mkdir('models/')

model.save("models/" + time + "-model.h5")

In [44]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 22972), started 4 days, 0:36:50 ago. (Use '!kill 22972' to kill it.)

In [59]:
img = keras.preprocessing.image.load_img(
    "example/mtb9.PNG", target_size=image_size
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

result = np.argmax(predictions[0])

p_list = { 0:"dress", 1:"outer", 2:"pants", 3:"skirt", 4:"suit", 5:"top"}

print(p_list[result])

dress
